In [10]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

import pyspark
import json

from pyspark import SparkConf , SparkContext
from pyspark.sql import SparkSession 

In [12]:

spark = SparkSession.builder.appName("data_json").getOrCreate()

df = spark.read.option("multiline" , "true").json("/home/kishanmahajan/Downloads/synthea_sample_data_fhir_r4_sep2019/fhir/Aaron697_Brekke496_2fa15bc7-8866-461a-9000-f739e425860a.json")


22/12/29 11:38:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [13]:
df.printSchema()

root
 |-- entry: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- fullUrl: string (nullable = true)
 |    |    |-- request: struct (nullable = true)
 |    |    |    |-- method: string (nullable = true)
 |    |    |    |-- url: string (nullable = true)
 |    |    |-- resource: struct (nullable = true)
 |    |    |    |-- abatementDateTime: string (nullable = true)
 |    |    |    |-- achievementStatus: struct (nullable = true)
 |    |    |    |    |-- coding: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- code: string (nullable = true)
 |    |    |    |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- active: boolean (nullable = true)
 |    |    |    |-- activity: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- detail: struct (nullable = true)
 |    |    |    |    |    |    |-- code

In [16]:
def flatten(df):
   # compute Complex Fields (Lists and Structs) in Schema   
    complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
    
    while len(complex_fields)!=0:
        col_name=list(complex_fields.keys())[0]
        print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))
        
        if (type(complex_fields[col_name]) == StructType):
            expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
            df=df.select("*", *expanded).drop(col_name)
            
            
        elif (type(complex_fields[col_name]) == ArrayType):    
            df=df.withColumn(col_name,explode_outer(col_name))
            
            
        complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
        
        
    return df

In [17]:
df_unflatten = flatten(df)

Processing :entry Type : <class 'pyspark.sql.types.ArrayType'>
Processing :entry Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_request Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_achievementStatus Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_activity Type : <class 'pyspark.sql.types.ArrayType'>
Processing :entry_resource_activity Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_address Type : <class 'pyspark.sql.types.ArrayType'>
Processing :entry_resource_address Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_addresses Type : <class 'pyspark.sql.types.ArrayType'>
Processing :entry_resource_addresses Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_billablePeriod Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_careTeam Type : <class 'pyspark.s

Processing :entry_resource_component_valueQuantity Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_contained_beneficiary Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_contained_payor Type : <class 'pyspark.sql.types.ArrayType'>
Processing :entry_resource_contained_payor Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_contained_performer Type : <class 'pyspark.sql.types.ArrayType'>
Processing :entry_resource_contained_performer Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_contained_requester Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_contained_subject Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_contained_type Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_diagnosis_diagnosisReference Type : <class 'pyspark.sql.types.StructType'>
Processing :entry_resource_diagnosis_type Type : <class 'pyspark.sql.typ

In [19]:
df_unflatten.printSchema()

root
 |-- resourceType: string (nullable = true)
 |-- type: string (nullable = true)
 |-- entry_fullUrl: string (nullable = true)
 |-- entry_request_method: string (nullable = true)
 |-- entry_request_url: string (nullable = true)
 |-- entry_resource_abatementDateTime: string (nullable = true)
 |-- entry_resource_active: boolean (nullable = true)
 |-- entry_resource_authoredOn: string (nullable = true)
 |-- entry_resource_birthDate: string (nullable = true)
 |-- entry_resource_created: string (nullable = true)
 |-- entry_resource_distinctIdentifier: string (nullable = true)
 |-- entry_resource_effectiveDateTime: string (nullable = true)
 |-- entry_resource_expirationDate: string (nullable = true)
 |-- entry_resource_gender: string (nullable = true)
 |-- entry_resource_id: string (nullable = true)
 |-- entry_resource_intent: string (nullable = true)
 |-- entry_resource_issued: string (nullable = true)
 |-- entry_resource_lifecycleStatus: string (nullable = true)
 |-- entry_resource_lotN

In [20]:
df_unflatten.show()

+------------+-----------+--------------------+--------------------+-----------------+--------------------------------+---------------------+-------------------------+------------------------+----------------------+---------------------------------+--------------------------------+-----------------------------+---------------------+--------------------+---------------------+---------------------+------------------------------+------------------------+------------------------------+-----------------------------------+--------------------+--------------------------------+-----------------------------+---------------------------------+----------------------------+----------------------+----------------------------+---------------------------+---------------------------+---------------------------+----------------------+---------------------+--------------------+-------------------+------------------+---------------------------+------------------------------+---------------------------+---